In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
from pathlib import Path
from pasna_analysis import Embryo, Experiment, utils
import matplotlib.pyplot as plt

experiment_name = '20240515'
exp_path = Path.cwd().parent.joinpath('data',experiment_name)
exp = Experiment(exp_path)

Visualize developmental time for a single embryo.

In [ ]:
activities = exp.activity()
lengths = exp.lengths()

i = 2

fig, ax = plt.subplots()

emb = Embryo(activities[i], lengths[i])
W = emb.developmental_time()
X = emb.vnc_length[:, 0]/60

ax.plot(X, W)
ax.set_title(f'VNC length - {emb.name}')
ax.set_xlabel('t(min)')
ax.set_ylabel('emb size:vnc size')

fig.tight_layout()

Visualize developmental time, for all embryos of the current experiment.

In [ ]:
lengths = exp.lengths()
activities = exp.activity()

fig, ax = plt.subplots(2, figsize=(10, 12))

for l, a in zip(lengths, activities):
    emb = Embryo(a, l)
    Y = emb.vnc_length[:, 1]
    W = emb.developmental_time()
    X = emb.vnc_length[:, 0]/60

    ax[0].plot(X, Y)
    ax[1].plot(X, W)


ax[0].set_title(f'VNC length - {exp.name}')
ax[0].set_xlabel('t(min)')
ax[0].set_ylabel('um')

fig.tight_layout()

Try to find the best fit for developmental time. 
Tried to use a linear regression and a log regression.
Log regression seems to perform a bit better.

In [ ]:
from scipy.optimize import curve_fit

def average(arr):
    return sum(arr)/len(arr)

def linear_func(x, a, b):
    return a*x + b

def log_func(x, a, b, c):
    return a*np.log(x + b) + c

def fit_data(x, y, func, p0=None):
    params, _ = curve_fit(func, x, y, p0=p0, maxfev=1_000_000)
    y_fit = func(x, *params)

    ss_res_lin = np.sum((y - y_fit) ** 2)
    ss_tot_lin = np.sum((y - np.mean(y)) ** 2)
    r2 = 1 - (ss_res_lin / ss_tot_lin)

    return y_fit, r2

lengths = exp.lengths()
activities = exp.activity()

fig, ax = plt.subplots(2, figsize=(10,12))
r2_lins = []
r2_logs = []
for act, length in zip(activities, lengths): 
    emb = Embryo(act, length)

    x = (emb.vnc_length[:, 0]/60).astype(np.float64)
    dev_time = emb.size / emb.vnc_length[:, 1].astype(np.float64)

    y_fit_lin, r2_lin = fit_data(x, dev_time, linear_func)
    y_fit_log, r2_log = fit_data(x, dev_time, log_func, p0=[2,2,2])
    r2_lins.append(r2_lin)
    r2_logs.append(r2_log)
    ax[0].plot(x, y_fit_lin)
    ax[1].plot(x, y_fit_log)

print(average(r2_lins))
print(average(r2_logs))

fig.supxlabel('t(min)')
fig.supylabel('developmental time')
ax[0].set_title('Linear fit')
ax[1].set_title('Log fit')
fig.tight_layout()